In [ ]:
import numpy as np
import igraph as ig

In [ ]:
class racing_and_refueling:
        
    def __init__(self, input_path):
        
        self.input_path = input_path
        
        #------------------------------------------------------------------------------------------------------------------------#
        def fix_line(line, mode):
            '''
            Inputs
                1. line: a line from the input text file that has to be fixed, i.e. strip, split, convert to integer
                2. mode: type of the line, either one of I (info), M (lines that describe a road), C (lines that describe a city
                in the racing route), or S (lines that describe a gas station)
            Outputs
                1. the contents of the given line in stripped (all modes), split (modes C and S), and integer (all modes) form
            '''
            if mode == "I":
                # input: "1 2 3 4 5\n"
                # output: (1, 2, 3, 4, 5)
                return tuple(map(int, line.strip().split()))
            elif mode == "M":
                # input: "1 2 3\n"
                    # -(strip)-> "1 2 3"
                    # -(split)-> ["1", "2", "3"]
                    # -(int)-> [1, 2, 3]
                    # -(lambda x: x - 1)-> easier to construct the adjacency matrix + refer to vertices and edges in igraph syntax
                    #                                                                 ... igraph works better with indices
                # output: [0, 1, 2]
                return tuple(map(lambda x: x - 1, list(map(int, line.strip().split()))))
            else:
                # ... igraph works better with indices
                return int(line.strip()) - 1
        
        #-------------------------------------------------------------------------------------------------------------------------#
        
        with open(self.input_path, "r") as input:
            # get the content of the text file into an iterator, named "lines"
            lines = input.readlines()

        # for the 1st line, use its corresponding information to store the values
        # (N, M, K, L, B), which will be used to construct the "for-loop" below
        (self.N, self.M, self.K, self.L, self.B) = fix_line(lines[0], "I")
        # increment the no_of_line counter, since nothing else has left to do for the 1st line

        # now that we have enough information about the size of the graph, i.e. N, we can
        # initialize it, as an adjacency matrix, with dimentions: (N, N) and data type: integer
        self.weighted_adjacency_matrix = np.zeros((self.N, self.N), dtype = int)
        # for the next M lines, use the corresponding information to populate the adjacency matrix
        for line in lines[1 : (self.M + 1)]:
            line = fix_line(line, "M")
            self.weighted_adjacency_matrix[line[0], line[1]] = self.weighted_adjacency_matrix[line[1], line[0]] = line[2] + 1

        self.racing_route = []
        # for the next K lines, use the corresponding information to construct the racing route
        for line in lines[(self.M + 1) : (self.M + self.K + 1)]:
            self.racing_route.append(fix_line(line, "C"))

        self.gas_stations = []
        # for the next M lines, use the corresponding information to construct the gas stations' locations
        for line in lines[(self.M + self.K + 1):]:
            self.gas_stations.append(fix_line(line, "S"))
            
    def __call__(self):
        
        #-----------------------------------------------------------------------------------------------------------------------------------------#
        def distance_racing_route(racing_route):
            '''
            Inputs
                1. racing_route
            Outputs
                1. total time to traverse the racing route
            '''
            
            return sum([self.weighted_adjacency_matrix[self.racing_route[i], self.racing_route[i + 1]] for i in range(len(self.racing_route) - 1)])
        #-----------------------------------------------------------------------------------------------------------------------------------------#
        
        def shortest_distance_from_gas_stations(graph, city, gas_stations):
            '''
            Inputs
                1. graph: the graph representation of the input
                2. city: a city of the racing route, from which the distance to the closest gas station will be calculated
                3. gas_stations: all of the available gas stations
            Outputs
                1. the minimum time required to travel from the closest gas station to the city of the racing route that
                was passed in the input argument: city
            '''
    
            # calculate the shortest *paths* from the city of the racing route to every reachable gas station
            # it is a list of B lists: [[path(S_1, C)], [path(S_2, C)], ..., [path(S_B, C)]], where each of the inner lists
            # correpsond to the shortest *path* (not distance, yet) from S_i to the specified city, C, of the racing route
            shortest_paths_from_gas_stations = graph.get_shortest_paths(city,
                to = gas_stations, weights = graph.es["weight"], output = "epath")

            # a list to store the shortest *distances* from S_1, ..., S_B to C
            shortest_distances = []
            # for each *path*: path(S_1, C), ..., path(S_B, C) ...
            for path in shortest_paths_from_gas_stations:
                # ... convert it to its *distance* equivalent: distance(S_1, C), ..., distance(S_B, C)
                distance = 0
                # if a path has length == 0, then the gas station is not reachable from the city
                # so, the if statement, below, protects from such cases
                if (len(path) > 0):
                    for edge in path:
                        distance += int(graph.es[edge]["weight"])
                    # this is *inside* the if clause, in order to prevent 0's from entering the shortest_distances
                    # 0's will cause bugs that are not traceable
                    shortest_distances.append(distance)

            # we return the shortest distance (in sec) from the closest gas station to the specified city
            # this is, simply, the minimum element of the shortest_distances list
            return min(shortest_distances)
        #-----------------------------------------------------------------------------------------------------------------------------------------#
        
        # construct the graph representation of the input file
        graph = ig.Graph.Weighted_Adjacency(self.weighted_adjacency_matrix.tolist())
    
        # calculate the distance (in sec) to traverse the racing route
        length_racing_route = distance_racing_route(racing_route = self.racing_route)

        # for each city of the racing route: C_2, ..., C_K-1
        # calculate the (shortest) distance to the nearest gas station
        # and store it into distance_gas_stations
        distance_gas_stations = []
        for city in self.racing_route[1:-1]:
            distance_gas_stations.append(shortest_distance_from_gas_stations(graph = graph,
                                                                             city = city,
                                                                             gas_stations = self.gas_stations))

        # now that we have the shortest distances from every city of the racing route to their nearest gas station
        # sort those shortest distances, in ascending order, ...
        distance_gas_stations = sorted(distance_gas_stations)

        # ... and add the sum of the L shortest distances to distance_racing_route
        # then, return the resulted quantity, which is the minimum time required to complete the race
        return length_racing_route + sum(distance_gas_stations[:self.L])

In [ ]:
racing_and_refueling_example = racing_and_refueling(input_path = "../input/test-case/input.txt")
output = racing_and_refueling_example()

In [ ]:
output